In [4]:
import sys
sys.path.append("../")
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from utils.inkml2img import convert_dir
from tqdm.auto import tqdm
from data.dataset import Im2LatexDataset
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from model.vit import ViT
import lightning as L
import wandb
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer, pre_tokenizers
from tokenizers.models import BPE
from model.decoder import Decoder, DecoderAttention, DecoderTransformerBlock
from model.model import Img2MathModel


In [5]:
img_dims = [256, 256]
data = Im2LatexDataset(path_to_data="../data/",
                       tokenizer="../data/tokenizer.json", img_dims=img_dims, batch_size=1, device=torch.device('mps'))
img, label = next(iter(data.train))
vocab_size = len(data.tokenizer.get_vocab())
enc_output = torch.randn(1, 512, 512)
model = Img2MathModel(512, 256, vocab_size, .75, img_dims, 16)
model(img[0], input_seq=label['input_ids'], trg_seq=label['input_ids'], mask=label['attention_mask'])

(tensor([[[ 0.0510,  0.8210,  0.2418,  ...,  0.6310,  1.0812, -0.3698],
          [-0.1525, -0.2930,  1.3171,  ...,  0.6546, -0.1515, -0.2718],
          [ 0.0415,  0.2966, -0.2223,  ...,  0.8965,  0.5535,  0.0965],
          ...,
          [ 1.2360,  0.0743, -0.3051,  ...,  1.1935,  1.2326,  0.2517],
          [-0.0595,  0.8408,  0.1121,  ...,  1.1305, -0.4039,  0.8401],
          [ 0.4998,  1.3164,  0.3612,  ..., -0.3101,  0.4249,  0.1453]]],
        grad_fn=<ViewBackward0>),
 tensor(7.8416, grad_fn=<NllLossBackward0>))

In [ ]:
logger = WandbLogger(project='img2math')

trainer = L.Trainer(limit_train_batches=10000, max_epochs=1, log_every_n_steps=20, deterministic=True,
                    logger=logger, accelerator='mps')

trainer.fit(model, data.train, data.test)
wandb.finish()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | encoder | ViT     | 8.9 M 
1 | decoder | Decoder | 14.9 M
------------------------------------
23.9 M    Trainable params
0         Non-trainable params
23.9 M    Total params
95.443    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

torch.Size([1])